<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Trodes/Exporting_Trodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import subprocess

In [39]:
reccording_source_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\Test_Emily'
os.chdir(reccording_source_dir)

dot_dat_export_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS'
os.chdir(dot_dat_export_dir)

rec_list=list()
for root, dirs, files in os.walk(reccording_source_dir):
    for f in files:
        if '.rec' in f:
            rec_list.append(os.path.join(root,f))
print(rec_list)


export_func_location = 'C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe'
filename = rec_list[0]

['\\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec']


In [32]:
""" HELP """
subprocess.run(export_func_location+' -h')

CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -h', returncode=0)

In [42]:
raw_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA'
full_cmd_raw = ' '.join([export_func_location, '-raw',
                    '-rec ', filename,
                    '-userawrefs', str(0),
                    '-outputdirectory', raw_dir])
print(full_cmd_raw)
subprocess.run(full_cmd_raw)

C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -raw -rec  \\hive.gladstone.internal\Huang-LFP\TabuenaLFP\Test_Emily\AngelfoodCake01\AngelfoodCake01.rec -userawrefs 0 -outputdirectory \\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA


CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -raw -rec  \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec -userawrefs 0 -outputdirectory \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\TRODE_EXPORTS\\RAW_DATA', returncode=0)

In [31]:
lfp_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\LFP_DATA'
full_cmd_lfp = ' '.join([export_func_location, '-lfp',
                         '-rec ', filename,
                         '-lfpoutputrate', str(5000),
                         '-uselfprefs', str(0),
                         '-uselfpfilters', str(0),
                         '-outputdirectory', lfp_dir])
report = subprocess.run(full_cmd_lfp)
print(report)

CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -lfp -rec  \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec -lfpoutputrate 5000 -uselfprefs 0 -uselfpfilters 0 -outputdirectory \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\TRODE_EXPORTS\\LFP_DATA', returncode=0)


In [30]:
spike_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\LFP_DATA'
full_cmd_spikeband = ' '.join([export_func_location, '-spikeband',
                         '-rec ', filename,
                         '-spikehighpass', str(600),
                         '-spikelowpass', str(6000),
                         '-usespikerefs', str(0),
                         '-usespikefilters', str(0),
                         '-outputdirectory', spike_dir])
report = subprocess.run(full_cmd_spikeband)
print(report)

CompletedProcess(args='C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/trodesexport.exe -spikeband -rec  \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\Test_Emily\\AngelfoodCake01\\AngelfoodCake01.rec -spikehighpass 600 -spikelowpass 6000 -usespikerefs 0 -usespikefilters 0 -outputdirectory \\\\hive.gladstone.internal\\Huang-LFP\\TabuenaLFP\\TRODE_EXPORTS', returncode=0)

In [ ]:
raw_dir = r'\\hive.gladstone.internal\Huang-LFP\TabuenaLFP\TRODE_EXPORTS\RAW_DATA'
full_cmd_raw = ' '.join([export_func_location, '-raw',
                         '-rec ', filename,
                         '-spikehighpass', str(600),
                         '-spikelowpass', str(6000),
                         '-usespikerefs', str(0),
                         '-usespikefilters', str(0),
                         '-outputdirectory', raw_dir])
report = subprocess.run(raw_dir)
print(report)

In [ ]:
os.chdir('C:/Users/dennis.tabuena/Desktop/_SPIKE_GADGETS/Trodes_2-5-1_Windows64/Trodes_2-5-1_Windows64/AngelfoodCake01.raw')
x = readTrodesExtractedDataFile('AngelfoodCake01.raw_group0.dat')

In [ ]:
#!/usr/bin/python
#Requires numpy to be installed. re and sys are defaults for python
#assumes python 2.7
import numpy as np
import re
from sys import argv
# Main function
def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        if f.readline().decode('ascii').strip() != '<Start settings>':
            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                line = line.decode('ascii').strip()
                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText


# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)



